In [41]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB052568,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325168,Herr,Gerd,Hagmaier,Male,...,2025-08-27 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
1,KK069910,Karma Resorts,Other,Past Guests,Karma Kandara,PF303975,Ms,Carlene,Bourke,Unknown,...,2025-08-25 00:00:00,3,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
2,KK079058,Karma Resorts,Other,Past Guests,Karma Kandara,PF327202,Miss,Nadine,Samir,Female,...,2025-08-26 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
3,KK067770,Karma Resorts,Other,Past Guests,Karma Kandara,PF299646,Mr.,Richard,Trotter,Male,...,2025-08-27 00:00:00,4,DIR,GSTAFF,COMP,6,0,0,Email and Phone,NO
4,KK079075,Karma Resorts,Other,Past Guests,Karma Kandara,PF226709,Miss,Sian Alexandra,Austin,Female,...,2025-08-27 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,KSM016741,Karma Resorts,Other,Past Guests,Karma St Martins,PF326186,Mr,Nick,George (Peter George Transport),Unknown,...,2025-08-24 00:00:00,2,DIR,WALKIN,RETAIL,2,0,0,Phone Only,NO
111,KSM016573,Karma Resorts,Other,Past Guests,Karma St Martins,PF324458,Miss,Evmorfia,Dimitriadi,Unknown,...,2025-08-26 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
112,KK066506,Karma Resorts,Other,Past Guests,Karma Kandara,PF297538,Mr,Simon,Pattinson,Male,...,2025-08-27 00:00:00,5,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
113,KK078922,Karma Resorts,Other,Past Guests,Karma Kandara,PF326808,Mr.,Ferdows,Adelyar,Male,...,2025-08-24 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO


In [43]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [44]:
df_analytic.fillna('empty', inplace=True)

In [45]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [46]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [47]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [48]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
22,KB051644,Karma Resorts,Other,Past Guests,Karma Bavaria,PF061147,Frau,Valentyna,Gernert,Female,...,2025-08-28 00:00:00,7,COMP,GER_OWE,COMP,4,0,0,Email and Phone,NO
24,KB046763,Karma Resorts,Other,Past Guests,Karma Bavaria,PF297268,Dr.,Johannes,Gansmeier,Unknown,...,2025-08-24 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
47,KK076950,Karma Resorts,Other,Past Guests,Karma Kandara,PF320081,Mr.,Shahril,Shahril Bin Mokhtar,Male,...,2025-08-26 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
48,KK076853,Karma Resorts,Other,Past Guests,Karma Kandara,PF320081,Mr.,Shahril,Shahril Bin Mokhtar,Male,...,2025-08-26 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
56,KB045005,Karma Resorts,Other,Past Guests,Karma Bavaria,PF289159,empty,Lee,Hobeika,Unknown,...,2025-08-31 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
58,KSH025185,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF315947,Ms,Getting Ready,Room,Female,...,2025-08-24 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email Only,NO
61,KSM016847,Karma Resorts,Other,Past Guests,Karma St Martins,PF318595,Mr,Angus Mcewen,Smith,Male,...,2025-08-26 00:00:00,1,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
63,KSH023454,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF306240,Mr,Nick,Chapman,Unknown,...,2025-08-25 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
93,KSM016766,Karma Resorts,Other,Past Guests,Karma St Martins,PF326329,Mr,Michael,Maltby,Unknown,...,2025-08-26 00:00:00,3,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
103,KSM016680,Karma Resorts,Other,Past Guests,Karma St Martins,PF325452,Mr,Ben,Gillet,Unknown,...,2025-08-28 00:00:00,5,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO


In [49]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB052568,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325168,Herr,Gerd,Hagmaier,Male,...,2025-08-27 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
1,KK069910,Karma Resorts,Other,Past Guests,Karma Kandara,PF303975,Ms,Carlene,Bourke,Unknown,...,2025-08-25 00:00:00,3,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
2,KK079058,Karma Resorts,Other,Past Guests,Karma Kandara,PF327202,Miss,Nadine,Samir,Female,...,2025-08-26 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
3,KK067770,Karma Resorts,Other,Past Guests,Karma Kandara,PF299646,Mr.,Richard,Trotter,Male,...,2025-08-27 00:00:00,4,DIR,GSTAFF,COMP,6,0,0,Email and Phone,NO
4,KK079075,Karma Resorts,Other,Past Guests,Karma Kandara,PF226709,Miss,Sian Alexandra,Austin,Female,...,2025-08-27 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,KSM016741,Karma Resorts,Other,Past Guests,Karma St Martins,PF326186,Mr,Nick,George (Peter George Transport),Unknown,...,2025-08-24 00:00:00,2,DIR,WALKIN,RETAIL,2,0,0,Phone Only,NO
101,KSM016573,Karma Resorts,Other,Past Guests,Karma St Martins,PF324458,Miss,Evmorfia,Dimitriadi,Unknown,...,2025-08-26 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
102,KK066506,Karma Resorts,Other,Past Guests,Karma Kandara,PF297538,Mr,Simon,Pattinson,Male,...,2025-08-27 00:00:00,5,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
103,KK078922,Karma Resorts,Other,Past Guests,Karma Kandara,PF326808,Mr.,Ferdows,Adelyar,Male,...,2025-08-24 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO


In [50]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [51]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [52]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
1,KK069910,Karma Resorts,Other,Past Guests,Karma Kandara,PF303975,Ms,Carlene,Bourke,Unknown,...,2025-08-25 00:00:00,3,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
17,KB052646,Karma Resorts,Other,Past Guests,Karma Bavaria,PF147772,Frau,Claudia,Goebl,Female,...,2025-08-25 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
19,KB052879,Karma Resorts,Other,Past Guests,Karma Bavaria,PF192683,Frau,Kathrin,Haas,Female,...,2025-08-25 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
21,KB050654,Karma Resorts,Other,Past Guests,Karma Bavaria,PF061147,Frau,Valentyna,Gernert,Female,...,2025-08-24 00:00:00,7,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
25,KB052944,Karma Resorts,Other,Past Guests,Karma Bavaria,PF246657,empty,Lisa,Rimschneider,Female,...,2025-08-28 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
47,KK076010,Karma Resorts,Other,Past Guests,Karma Kandara,PF076064,Mrs,Galina,Letunovskaya,Female,...,2025-08-25 00:00:00,8,HOUSE_USE,V_OWNER_NRB,COMP,2,1,0,Email and Phone,NO


In [53]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [54]:
count_data_akhir = df_analytic.shape[0]

In [55]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB046556,Karma Resorts,Other,Past Guests,Karma Bavaria,PF326081,empty,empty,Luca Kochendorfer,Unknown,...,2025-08-24 00:00:00,1,OTA,OTA,RETAIL,6,0,0,Email Only,NO
1,KB052029,Karma Resorts,Other,Past Guests,Karma Bavaria,PF322306,Frau,Tanya,Armbruster,Female,...,2025-08-24 00:00:00,3,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
2,KSH023222,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF304756,Mr,Mike,Revill,Male,...,2025-08-24 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
3,KB052559,Karma Resorts,Other,Past Guests,Karma Bavaria,PF134385,Mr,Derek,De Jager,Male,...,2025-08-24 00:00:00,14,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
4,KB049092,Karma Resorts,Other,Past Guests,Karma Bavaria,PF308213,Mr,Eigner,Daniel,Unknown,...,2025-08-24 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,KB051938,Karma Resorts,Other,Past Guests,Karma Bavaria,PF321775,empty,Andrea,Seyferle,Unknown,...,2025-08-31 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
95,KB050169,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312397,empty,Tatjana,Hepp,Unknown,...,2025-08-31 00:00:00,2,OTA,OTA,RETAIL,6,0,0,Phone Only,NO
96,KB051389,Karma Resorts,Other,Past Guests,Karma Bavaria,PF318817,empty,Axel,Trautmann,Unknown,...,2025-08-31 00:00:00,1,OTA,OTA,RETAIL,2,1,0,Phone Only,NO
97,KB045004,Karma Resorts,Other,Past Guests,Karma Bavaria,PF289159,empty,Lee,Hobeika,Unknown,...,2025-08-31 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [56]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [57]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_7720\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [58]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB046556,Karma Resorts,Other,Past Guests,Karma Bavaria,PF326081,,,Luca Kochendorfer,Unknown,...,2025-08-24,1,OTA,OTA,RETAIL,6,0,0,Email Only,NO
1,KB052712,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325890,Dr,Blanka,Szost-Ouk,Unknown,...,2025-08-24,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
2,KSH019750,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF284361,Mrs,Tracy,Lee,Female,...,2025-08-24,2,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
3,KK075877,Karma Resorts,Other,Past Guests,Karma Kandara,PF318151,Mr,Oapah,Katsande Nyabinde,Male,...,2025-08-24,5,OTA,OTA,RETAIL,9,0,0,Phone Only,NO
4,KSH022942,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF303154,Mr,Philip,Carney,Male,...,2025-08-24,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,KB045004,Karma Resorts,Other,Past Guests,Karma Bavaria,PF289159,,Lee,Hobeika,Unknown,...,2025-08-31,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
95,KB051938,Karma Resorts,Other,Past Guests,Karma Bavaria,PF321775,,Andrea,Seyferle,Unknown,...,2025-08-31,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
96,KB050169,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312397,,Tatjana,Hepp,Unknown,...,2025-08-31,2,OTA,OTA,RETAIL,6,0,0,Phone Only,NO
97,KB051389,Karma Resorts,Other,Past Guests,Karma Bavaria,PF318817,,Axel,Trautmann,Unknown,...,2025-08-31,1,OTA,OTA,RETAIL,2,1,0,Phone Only,NO


In [59]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 115
Jumlah Data Yang Duplicate : 10
Jumlah Data Setelah Hapus Duplicate : 105
Jumlah Data Yang Termasuk Member: 6
Jumlah Data Setelah Hapus Member : 99
Jumlah Data Akhir : 99


In [60]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)